# MED Lab 2023/2024 - Projekt Zaliczeniowy
Zadaniem tego projektu indywidualnego jest zaliczenie laboratorium z MED.

Projekt należy wykonać **bez użycia jakichkolwiek innych bibliotek/pakietów, niż te, które zostały podane w pakietach podstawowych**

Poniższe zadania mają potwierdzić posiadane przez Państwa umiejętności budowania algorytmów uczenia maszynowego i  modelowania regresji na przykładzie danych zawierających ceny nieruchomości znajdujących się w pliku "domy.csv". W zadaniu należy wykorzystać wiedzę poznaną na laboratorium i wykładzie w tym metody regularyzacji, metody wyboru i tworzenia nowych zmiennych, określenie jakości modelu.

**Projekt należy wgrać na ISOD do 11.12.2023**

Z projektu można zdobyć 50 pkt. Za każde zadanie można dostać od 0 do 10 pkt.

Można korzystać z dowolnych źródeł.

### Uwaga !!!
Każda osoba pracuje na inaczej podzielonych danych zależnych od numeru indeksu, więc podanie wniosków nieadekwatnych do otrzymanych wyników będzie uznane za próbę ściągania i karane niezaliczeniem projektu, a co za tym idzie przedmiotu.

In [23]:
# Wczytanie pakietów podstawowych
import numpy as np
import pandas as pd

In [24]:
# Wczytanie danych
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
domy = pd.read_csv("domy.csv", sep=',', dtype=dtype_dict)
domy

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,0263000018,20140521T000000,360000.0,3.0,2.50,1530.0,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530.0,1509.0
21609,6600060120,20150223T000000,400000.0,4.0,2.50,2310.0,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830.0,7200.0
21610,1523300141,20140623T000000,402101.0,2.0,0.75,1020.0,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020.0,2007.0
21611,0291310100,20150116T000000,400000.0,3.0,2.50,1600.0,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410.0,1287.0


### Podział danych zostanie dokonany zależnie od numeru indeksu

In [25]:
from sklearn.model_selection import train_test_split
# !!!Jako random_state podaj swój numer indeksu!!!
numer_indeksu = 319031
ceny = domy['price']
remove = ['id', 'date', 'zipcode', 'price']
domy.drop(columns = remove, inplace=True)

In [26]:
domy_train, domy_test, ceny_train, ceny_test = train_test_split(domy, ceny, test_size=0.33, random_state=numer_indeksu)
cechy = domy.columns.values
cechy 

array(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'], dtype=object)

## Zadanie 1
Dokonaj analizy posiadanych zmiennych. Zakresy zmienności, korelacje, skośność. Dane nieużyteczne jak np. id można odrzucić. 

Zaproponuj 5 nowych cech będących wynikiem przeprowadzonych analiz lub doświadczeń.

In [27]:
#Statystyki opisowe
domy_train.describe()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
count,14480.000000,14480.000000,14480.000000,1.448000e+04,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000,14480.000000
mean,3.378660,2.116816,2081.351934,1.509254e+04,1.495684,0.007390,0.237569,3.407735,7.663467,1790.076036,291.275898,1971.193094,82.006077,47.559629,-122.214093,1988.008494,12682.857044
std,0.937983,0.771487,915.780056,4.245653e+04,0.538777,0.085647,0.773944,0.649860,1.170141,826.198942,441.579342,29.456917,396.176931,0.138681,0.139165,682.009464,27870.010426
min,0.000000,0.000000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,47.155900,-122.515000,399.000000,748.000000
25%,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1200.000000,0.000000,1952.000000,0.000000,47.469800,-122.327250,1490.000000,5100.000000
50%,3.000000,2.250000,1910.000000,7.581000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,47.572000,-122.229000,1850.000000,7620.000000
75%,4.000000,2.500000,2550.000000,1.057550e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2220.000000,560.000000,1997.000000,0.000000,47.677525,-122.125000,2370.000000,10031.000000
max,33.000000,8.000000,12050.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,8860.000000,4820.000000,2015.000000,2015.000000,47.777600,-121.319000,6210.000000,871200.000000


In [28]:
#Korelacja
domy_train.corr()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
bedrooms,1.000000,0.509085,0.571966,0.034976,0.172630,-0.000444,0.082714,0.021685,0.358501,0.475313,0.296870,0.149892,0.028009,-0.000732,0.133924,0.390744,0.026708
bathrooms,0.509085,1.000000,0.756251,0.086904,0.500832,0.063500,0.200648,-0.125935,0.666136,0.685239,0.286282,0.502863,0.064354,0.031780,0.230997,0.572135,0.084696
sqft_living,0.571966,0.756251,1.000000,0.165220,0.352817,0.108736,0.290810,-0.065493,0.763128,0.876445,0.434038,0.319657,0.070961,0.061836,0.248326,0.754511,0.176722
sqft_lot,0.034976,0.086904,0.165220,1.000000,-0.011517,0.022671,0.075381,-0.008077,0.102565,0.172373,0.020134,0.043911,0.011260,-0.084276,0.234874,0.138386,0.701029
floors,0.172630,0.500832,0.352817,-0.011517,1.000000,0.020897,0.035254,-0.266793,0.456602,0.524179,-0.249046,0.489340,0.015345,0.052744,0.130614,0.279039,-0.013006
waterfront,-0.000444,0.063500,0.108736,0.022671,0.020897,1.000000,0.397581,0.015353,0.075123,0.070869,0.092909,-0.029502,0.091437,-0.008239,-0.033261,0.090550,0.030053
view,0.082714,0.200648,0.290810,0.075381,0.035254,0.397581,1.000000,0.040975,0.254312,0.171490,0.282245,-0.051798,0.114795,0.012891,-0.076036,0.277963,0.072005
condition,0.021685,-0.125935,-0.065493,-0.008077,-0.266793,0.015353,0.040975,1.000000,-0.152414,-0.161117,0.165628,-0.360368,-0.059167,-0.018520,-0.105463,-0.100648,-0.004680
grade,0.358501,0.666136,0.763128,0.102565,0.456602,0.075123,0.254312,-0.152414,1.000000,0.755039,0.169947,0.448880,0.027880,0.121298,0.204214,0.711864,0.106381
sqft_above,0.475313,0.685239,0.876445,0.172373,0.524179,0.070869,0.171490,-0.161117,0.755039,1.000000,-0.053372,0.423789,0.035862,0.006278,0.351005,0.730982,0.185441


Można zauważyć silną dodatnią korelacje pomiędzy sqft_above i sqft_living, grade i sqft_living, sqft_living i bathrooms. Oraz można zauważyć największą korelacje ujemną pomiędzy yr_built i condition.

In [29]:
#Skosnosc
domy_train.skew()

bedrooms          2.647646
bathrooms         0.520694
sqft_living       1.463887
sqft_lot         13.763076
floors            0.601129
waterfront       11.504867
view              3.375605
condition         1.053795
grade             0.752418
sqft_above        1.449906
sqft_basement     1.592531
yr_built         -0.479360
yr_renovated      4.624694
lat              -0.485744
long              0.821547
sqft_living15     1.078193
sqft_lot15       10.216488
dtype: float64

Widzimy dużą dodanią skośność dla sqft_lot, waterfront i sqft_lot15  oraz największą ujemną skośność dla lat.

In [30]:
#Nowe cechy
domy_train['bedrooms_squared'] = domy_train['bedrooms'].apply(lambda x: x**2)
domy_test['bedrooms_squared'] = domy_test['bedrooms'].apply(lambda x: x**2)

domy_train['bed_bath_rooms'] = domy_train['bedrooms'] * domy_train['bathrooms']
domy_test['bed_bath_rooms'] = domy_test['bedrooms'] * domy_test['bathrooms']

domy_train['lat_plus_long'] = domy_train['lat'] + domy_train['long']
domy_test['lat_plus_long'] = domy_test['lat'] + domy_test['long']

domy_train['condition_grade'] = domy_train['condition'] * domy_train['grade']
domy_test['condition_grade'] = domy_test['condition'] * domy_test['grade']

domy_train['log_sqft_living'] = domy_train['sqft_living'].apply(lambda x: np.log(x))
domy_test['log_sqft_living'] = domy_test['sqft_living'].apply(lambda x: np.log(x))

* bedrooms_squared - zwiększenie separację między nielicznymi sypialniami i wieloma sypialniami
* bed_bath_rooms - funkcja interakcji
* lat_plus_long
* condition_grade - funkcja interakcji, wynik wysoki gdy wartości obu będą wysokie
* log_sqft_living -  zbliżenie dużych wartości i rozłożenie małych wartości

## Zadanie 2
Napisz metodę regresji krokowej w tył i za jej pomocą wybierz 5 najbardziej wartościowych cech.

In [31]:
features = domy_train.columns.tolist()

while len(features) > 5:
    mse_values = []
    
    for feature in features:
        tmp_features = features
        mask = np.array(features) != feature
        tmp_features = np.array(tmp_features)[mask].tolist()
        
        pinw_tmp = np.linalg.pinv(domy_train[tmp_features])
        model_tmp = np.dot(pinw_tmp, ceny_train)
        
        mse = np.mean((np.dot(domy_train[tmp_features], model_tmp) - ceny_train) ** 2)
        mse_values.append(mse)

    min_mse_feature = features[np.argmin(mse_values)]
    
    features.remove(min_mse_feature)

print("Wybrane cechy:", features)

Wybrane cechy: ['sqft_living', 'condition', 'lat', 'condition_grade', 'log_sqft_living']


## Zadanie 3
Dokonaj normalizacji cech (wszystkie podane + 5 zaproponowanych w Zadaniu 1) i przy pomocy metody Lasso wybierz 5 najbardziej wartościowych cech. (Zadanie wymaga napisania metody optymalizacji dla regularyzacji Lasso)

!!! Przy przekształceniach unikaj korzystania z informacji zawartej w zbiorze walidacyjnym i testowym !!!

In [32]:
#Normalizacja cech
domy_train = (domy_train - domy_train.mean(axis= 0)) / domy_train.std(axis= 0)

In [33]:
class LassoRegression() : 
      
    def __init__( self, learning_rate, iterations, lamb ) :        
        self.learning_rate = learning_rate         
        self.iterations = iterations       
        self.lamb = lamb 
                    
    def fit( self, X, Y ) : 
        self.m, self.n = X.shape  
        self.W = np.zeros( self.n )     
        self.b = 0    
        self.X = X        
        self.Y = Y 
                  
        for i in range( self.iterations ) :           
            self.update_weights()         
        return self
      
    def update_weights( self ) :           
        Y_pred = self.predict( self.X ) 
        dW = np.zeros( self.n ) 
          
        for j in range( self.n ) :       
            if self.W[j] > 0 :               
                dW[j] = ( - ( 2 * ( self.X[:, j] ).dot( self.Y - Y_pred ) ) + self.lamb ) / self.m        
            else :                  
                dW[j] = ( - ( 2 * ( self.X[:, j] ).dot( self.Y - Y_pred ) ) - self.lamb ) / self.m 
        
        db = - 2 * np.sum( self.Y - Y_pred ) / self.m  
        self.W = self.W - self.learning_rate * dW    
        self.b = self.b - self.learning_rate * db 
          
        return self
      
    def predict( self, X ) :    
        return X.dot( self.W ) + self.b 

In [50]:
model = LassoRegression( iterations = 1000, learning_rate = 0.01, lamb = 266000000000) 
model.fit(domy_train.to_numpy(), ceny_train.to_numpy())

all_features = domy_train.columns.tolist()
for feature, coef in zip(all_features, model.W):
    if coef > 0:
        print(f'Waga dla cechy {feature} wynosi {coef}')

Waga dla cechy sqft_living wynosi 18458.092546201748
Waga dla cechy sqft_above wynosi 105753.45298376668
Waga dla cechy sqft_living15 wynosi 140442.48965183902
Waga dla cechy bed_bath_rooms wynosi 88373.5916296769
Waga dla cechy condition_grade wynosi 162753.0414406819


## Zadanie 4
Dla 5 wybranych w Zadaniu 3 cech dokonaj doboru współczynnika regularyzacji dla metody grzbietowej.
(Zadanie wymaga napisania dowolnej metody optymalizacji dla regularyzacji Ridge)

In [51]:
class RidgeRegression():   
    def __init__(self, learning_rate, iterations, alpha) :       
        self.learning_rate = learning_rate         
        self.iterations = iterations         
        self.alpha = alpha 
                   
    def fit(self, X, Y) :         
        self.m, self.n = X.shape        
        self.W = np.zeros( self.n ) 
        self.b = 0        
        self.X = X         
        self.Y = Y 
     
        for i in range( self.iterations ) :             
            self.update_weights()             
        return self
       
    def update_weights( self ) :            
        Y_pred = self.predict( self.X ) 
                
        dW = ( - ( 2 * ( self.X.T ).dot( self.Y - Y_pred ) ) + ( 2 * self.alpha * self.W ) ) / self.m      
        db = - 2 * np.sum( self.Y - Y_pred ) / self.m  
             
        self.W = self.W - self.learning_rate * dW     
        self.b = self.b - self.learning_rate * db         
        return self
       
    def predict( self, X ) :     
        return X.dot( self.W ) + self.b 

In [77]:
def k_fold_cross_validation(X, y, k_folds, model):
    fold_size = len(X) // k_folds
    scores = []

    for fold in range(k_folds):
        start_idx = fold * fold_size
        end_idx = (fold + 1) * fold_size if fold < k_folds - 1 else len(X)

        X_train = pd.concat([X.iloc[:start_idx], X.iloc[end_idx:]])
        y_train = pd.concat([y.iloc[:start_idx], y.iloc[end_idx:]])

        X_test = X.iloc[start_idx:end_idx]
        y_test = y.iloc[start_idx:end_idx]

        model.fit(X_train.values, y_train.values)
        y_pred = model.predict(X_test.values)

        mse = np.mean((y_pred - y_test.values) ** 2)
        scores.append(mse)

    mean_score = np.mean(scores)
    return mean_score

In [80]:
ridge_features = ['sqft_living', 'sqft_above', 'sqft_living15', 'bed_bath_rooms', 'condition_grade']
lambdas = [0.01]
while (l_neu := lambdas[-1]) < 50:
    lambdas.append(l_neu * 2)

best_lambda = None
best_mse = float('inf')

for a in lambdas:
    ridge_model = RidgeRegression(learning_rate=0.01, iterations=1000, alpha=a)
    mse = k_fold_cross_validation(domy_train[ridge_features], ceny_train, 5, model)

    if mse < best_mse:
        best_mse = mse
        best_lambda = alpha

print("Best alpha:", best_lambda)

Best alpha: 81.92


## Zadanie 5
Wytrenuj 5 modeli:
1. Wykorzystujący wszystkie cechy dane w zbiorze.
2. Wykorzystujący wszystkie cechy dane w zbiorze + 5 cech zaproponowanych.
3. Wykorzystujący 5 cech wybranych regresją krokową w tył.
4. Wykorzystujący tylko cechy wybrane metodą Lasso.
5. Wykorzystujące cechy wybrane w metodą Lasso i poddane regularyzacji grzbietowej dla najlepszego/wybranego współczynnika regularyzacji.

Porównaj średni błąd uczenia i testowania dla tych 5 modeli. Oceń, który model jest najlepszy.

In [55]:
def model_train(X_train, y_train):
    # Wylicz współczynniki modelu wykorzystując pseudoodwrotność
    pinw = np.linalg.pinv(X_train)
    model = np.dot(pinw,y_train)
    return model

def predict_output(X, w):
    return np.dot(X, w)

def policz_SSE(model, data, outcome):
    # Wylicz przewidywaną wartość
    pred_val = predict_output(data, model)
    # Policz błąd bezwzglęny
    abs_err = outcome - pred_val
    # Podnieś błędy do kwadratu i zsumuj je lub skorzystaj z mnożenia wektorów.
    SSE = np.sum(abs_err ** 2)
    return SSE   

In [57]:
#Wykorzystujący wszystkie cechy dane w zbiorze
model_1_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long',
       'sqft_living15', 'sqft_lot15']
model_1 = model_train(domy_train[model_1_features], ceny_train)
SSE1_train= policz_SSE(model_1, domy_train[model_1_features], ceny_train)
SSE1_test= policz_SSE(model_1, domy_test[model_1_features], ceny_test)
print(f'Średni błąd uczenia modelu 1: {SSE1_train}' )
print(f'Średni błąd testowania modelu 1: {SSE1_test}' )

Średni błąd uczenia modelu 1: 4856478586916480.0
Średni błąd testowania modelu 1: 5.103700701088104e+20


In [59]:
#Wykorzystujący wszystkie cechy dane w zbiorze + 5 cech zaproponowanych
model_2_features = model_1_features + ['bedrooms_squared', 'bed_bath_rooms', 'lat_plus_long', 'condition_grade', 'log_sqft_living']
model_2 = model_train(domy_train[model_2_features], ceny_train)
SSE2_train= policz_SSE(model_2, domy_train[model_2_features], ceny_train)
SSE2_test= policz_SSE(model_2, domy_test[model_2_features], ceny_test)
print(f'Średni błąd uczenia modelu 2: {SSE2_train}' )
print(f'Średni błąd testowania modelu 2: {SSE2_test}' )

Średni błąd uczenia modelu 2: 546756054584450.2
Średni błąd testowania modelu 2: 2.3416033576602833e+41


In [60]:
#Wykorzystujący 5 cech wybranych regresją krokową w tył.
model_3_features = ['sqft_living', 'condition', 'lat', 'condition_grade', 'log_sqft_living']
model_3 = model_train(domy_train[model_3_features], ceny_train)
SSE3_train= policz_SSE(model_3, domy_train[model_3_features], ceny_train)
SSE3_test= policz_SSE(model_3, domy_test[model_3_features], ceny_test)
print(f'Średni błąd uczenia modelu 3: {SSE3_train}' )
print(f'Średni błąd testowania modelu 3: {SSE3_test}' )

Średni błąd uczenia modelu 3: 4920611565858406.0
Średni błąd testowania modelu 3: 7.23029283657227e+21


In [61]:
#Wykorzystujący tylko cechy wybrane metodą Lasso
model_4_features = ['sqft_living', 'sqft_above', 'sqft_living15', 'bed_bath_rooms', 'condition_grade']
model_4 = model_train(domy_train[model_4_features], ceny_train)
SSE4_train= policz_SSE(model_4, domy_train[model_4_features], ceny_train)
SSE4_test= policz_SSE(model_4, domy_test[model_4_features], ceny_test)
print(f'Średni błąd uczenia modelu 4: {SSE4_train}' )
print(f'Średni błąd testowania modelu 4: {SSE4_test}' )

Średni błąd uczenia modelu 4: 5179979220334590.0
Średni błąd testowania modelu 4: 2.2970430452118685e+21


In [86]:
#Wykorzystujące cechy wybrane w metodą Lasso i poddane regularyzacji grzbietowej dla najlepszego/wybranego współczynnika regularyzacji.
a = 81.92

ridge_model = RidgeRegression(learning_rate=0.01, iterations=1000, alpha=a)
ridge_model.fit(domy_train[model_4_features], ceny_train)

train_error = policz_SSE(ridge_model.W, domy_train[model_4_features], ceny_train)

test_error = policz_SSE(ridge_model.W, domy_test[model_4_features], ceny_test)

print("Średni błąd uczenia modelu 5:", train_error)
print("Średni błąd testowania modelu 5:", test_error)

Średni błąd uczenia modelu 5: 5181591057464654.0
Średni błąd testowania modelu 5: 2.1071477800856563e+21


Wyniki dla modelów 2, 4, 5 nieznacznie się różnie, minimalnie lepiej od nich wychodzi model 5 wykorzystujący cechy wybrane metodą Lasso i poddane regularyzacji grzbietowej, ponieważ ma najmniejszy błąd testowania modelu.